In [ ]:
TF_RECORD_FILE = '/media/commaai-03/Data/dataset/segment-1208303279778032257_1360_000_1380_000_with_camera_labels.tfrecord'

In [ ]:
import os
import math
import tensorflow as tf
import numpy as np

In [ ]:
# For running session without tensor running.
tf.enable_eager_execution()

# Waymo open dataset Modules
# waymo_open_dataset: pip3 install waymo-open-dataset
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import frame_utils

In [ ]:
def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames=filenames, 
                                      compression_type='')
    frames = []
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)
    
    return frames

In [ ]:
frames = load_dataset(TF_RECORD_FILE)

In [ ]:
frames[0].context

### Waymo Open Dataset Format

#### 1. Lidar Data:

- Top(mid-range lidar)
- Front
- Side Left
- Side Right
- Rear

**Describe:** Each lidar provides two range images, one for each of the two strongest returns.

**Format:**

- Basic: [range, lidar intensity, lidar elongation, is_in_nlz(1 in but -1 not in)]
- Additional: [camera_name, x, y, camera name of 2nd projection, x, y] (**Note:** x: axis along image width, y: axis along image height, camera name of 2nd projection: set to UNKNOWN if no projection.)

---

#### 2. Camera Data:

- Front
- Front Left
- Front Right
- Side Left
- Side Right

In [ ]:
(range_images, camera_projections, range_image_top_pose) = \
    frame_utils.parse_range_image_and_camera_projection(frames[0])

In [ ]:
print(len(range_images))
print(len(camera_projections))
print(min(range_image_top_pose.data[:]))

In [ ]:
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frames[0], 
    range_images, 
    camera_projections, 
    range_image_top_pose)
# points [B, x, y, r]

In [ ]:
import cv2

#depth_data = np.array(points[1])
print(np.min(points[:][:][2]))
print(points[0][0])

for i in range(5):
    print(cp_points[i][0])
#cv2.imwrite('/home/commaai-03/Test/tmp/1.pgm', depth_data)

In [ ]:
#np.average(depth_data)